# Scrapper Etherscan- Get Name Tag of Known Addresses

## Prepare Notebook

In [1]:
# Import Libraries
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import warnings
warnings.filterwarnings(action='once')

In [ ]:
#Make API request to Etherscan
url = 'https://etherscan.io/accounts/%s?ps=100'%(1) # Initiate with the first page of accounts with 100 records
req = Request(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})   # Etherscan restricts the access for Webscrapping, reason why headers needs to be stated
response = urlopen(req, timeout=50).read() # Read response
page_soup = soup(response, "html.parser") # Parse HTML response
Transfers_info_table_1 = page_soup.find("table", {"class": "table table-hover"}) # Find the table in the HTML where is the information required
df=pd.read_html(str(Transfers_info_table_1))[0] # Save information in dataframe

#Iterate over the next pages to get the information of the top 10,000 accounts by balance
for i in range(2,101):
    url = 'https://etherscan.io/accounts/%s?ps=100'%(i) # State URL with current in iteration
    req = Request(url, headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}) # Etherscan restricts the access for Webscrapping, reason why headers needs to be stated
    response = urlopen(req, timeout=50).read() # Read response
    page_soup = soup(response, "html.parser") # Parse HTML response
    Transfers_info_table_1 = page_soup.find("table", {"class": "table table-hover"}) # Find the table in the HTML where is the information required
    df2=pd.read_html(str(Transfers_info_table_1))[0] # Save information in dataframe
    df=df.append(df2) # Append data to df
    time.sleep(2) # Etherscan restricts the access if the requests are perform too fast. The function delays the code by two seconds per iteration

In [5]:
# Print table with the information of the top 10,000 addresses from Etherscan
df

,Rank,Address,Name Tag,Balance,Percentage,Txn Count
0,1,0x00000000219ab540356cbb839cbe05303d7705fa,Eth2 Deposit Contract,"13,170,549.000069 Ether",10.98835551%,209882
1,2,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,"4,277,770.66721275 Ether",3.56899814%,8996540
2,3,0xda9dfa130df4de4673b89022ee50ff26f6ea73cf,Kraken 13,"2,113,030.00445455 Ether",1.76292764%,68
3,4,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,Binance 7,"1,996,008.27992735 Ether",1.66529494%,1098
4,5,0x0716a17fbaee714f1e6ab0f9d59edbc5f09815c0,NaN,"1,950,476.62187696 Ether",1.62730730%,24
...,...,...,...,...,...,...
95,9996,0xd2fc0f36529476cff219d03007958760c8f69e42,NaN,600.23061354 Ether,0.00050078%,1
96,9997,0x427e929f2fd2a741aa007442ec80bb74d6ee2a5a,NaN,600.22444736 Ether,0.00050077%,48
97,9998,0x1b4d65e1e2c7d45f0539cd06c2cd36743db0e83a,NaN,600.20224059 Ether,0.00050076%,4
98,9999,0x66e5555ec62a9ddf8efa76d2c1840f2474a0e8ac,NaN,600.19468433 Ether,0.00050075%,4


In [6]:
# Save and print table with only the addresses that have Name Tag
df_nametag=df[~df['Name Tag'].isna()]
df_nametag

,Rank,Address,Name Tag,Balance,Percentage,Txn Count
0,1,0x00000000219ab540356cbb839cbe05303d7705fa,Eth2 Deposit Contract,"13,170,549.000069 Ether",10.98835551%,209882
1,2,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,"4,277,770.66721275 Ether",3.56899814%,8996540
2,3,0xda9dfa130df4de4673b89022ee50ff26f6ea73cf,Kraken 13,"2,113,030.00445455 Ether",1.76292764%,68
3,4,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,Binance 7,"1,996,008.27992735 Ether",1.66529494%,1098
6,7,0x742d35cc6634c0532925a3b844bc454e4438f44e,Bitfinex 2,"1,341,122.36792507 Ether",1.11891534%,17185
...,...,...,...,...,...,...
10,9611,0x136d841d4bece3fc0e4debb94356d8b6b4b93209,InsurAce Protocol: Stakers Pool V2,627.70620349 Ether,0.00052370%,61
32,9833,0xe28e72fcf78647adce1f1252f240bbfaebd63bcc,Augur: Token Sale,610.002 Ether,0.00050893%,592
59,9860,0xcc120361cb6ded8c3611460a59176512d8758dbb,AyeZee: Copium Wallet,607.64890388 Ether,0.00050697%,384
16,9917,0x6a159503e49ddb8e49659d6cacf5236ac4213d86,BitDAO: ByBit Contribution,603.06177864 Ether,0.00050314%,69


In [7]:
# Export to CSV tables generated
df.to_csv("Etherscan_Top10000_Addresses.csv",index=False)
df_nametag.to_csv("Etherscan_NameTag_Addresses.csv",index=False)
